In [2]:
import os
import numpy as np 
import pandas as pd

In [3]:
path = r"C:\Users\piush\Desktop\World Bank Poverty\submissions\a_sub"

all_files = os.listdir(path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(path, f), index_col=[0,1]) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_poor_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,country,is_poor_0,is_poor_1,is_poor_2,is_poor_3,is_poor_4,is_poor_5,is_poor_6,is_poor_7,is_poor_8,is_poor_9,is_poor_10,is_poor_11,is_poor_12
0,418,A,0.923226,0.923226,0.994279,0.986550,0.982553,0.923226,0.977091,0.964370,0.997867,0.987687,0.992923,0.997882,0.983616
1,41249,A,0.000348,0.000348,0.000093,0.000076,0.000106,0.000348,0.000025,0.000070,0.000070,0.000141,0.000100,0.000046,0.000123
2,16205,A,0.797718,0.797718,0.996676,0.993071,0.993697,0.797718,0.908619,0.941512,0.973487,0.995622,0.987063,0.998779,0.979196
3,97501,A,0.000348,0.000348,0.000093,0.000076,0.000106,0.000348,0.000025,0.000070,0.000070,0.000141,0.000100,0.000046,0.000123
4,67756,A,0.992093,0.992093,0.999413,0.990447,0.924967,0.992093,0.998898,0.995074,0.998306,0.992904,0.999373,0.998777,0.996839


In [5]:
concat_sub.iloc[:,2:15].head(2)

,is_poor_0,is_poor_1,is_poor_2,is_poor_3,is_poor_4,is_poor_5,is_poor_6,is_poor_7,is_poor_8,is_poor_9,is_poor_10,is_poor_11,is_poor_12
0,0.923226,0.923226,0.994279,0.986550,0.982553,0.923226,0.977091,0.96437,0.997867,0.987687,0.992923,0.997882,0.983616
1,0.000348,0.000348,0.000093,0.000076,0.000106,0.000348,0.000025,0.00007,0.000070,0.000141,0.000100,0.000046,0.000123


In [6]:
concat_sub['poor'] = concat_sub.iloc[:,2:15].mean(axis = 1)

In [7]:
a = concat_sub[['id','country','poor']]

In [8]:
a.to_csv("a_1.csv")

In [ ]:
# get the data fields ready for stacking
concat_sub['is_poor_max'] = concat_sub.iloc[:, 2:18].max(axis=1)
concat_sub['is_poor_min'] = concat_sub.iloc[:, 2:18].min(axis=1)
concat_sub['is_poor_mean'] = concat_sub.iloc[:, 2:18].mean(axis=1)
concat_sub['is_poor_median'] = concat_sub.iloc[:, 2:18].median(axis=1)

In [ ]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [ ]:
concat_sub.head()

### Change the column names 

###### Mean Stacking

In [ ]:
concat_sub['is_poor'] = concat_sub['is_poor_mean']
concat_sub[['id','country', 'is_poor']].to_csv('stack_mean.csv', 
                                        index=False, float_format='%.6f')

###### Median Stacking

In [ ]:
concat_sub['is_poor'] = concat_sub['is_poor_median']
concat_sub[['id','country', 'is_poor']].to_csv('stack_median.csv', 
                                        index=False, float_format='%.6f')

###### PushOut + Median Stacking
Pushout strategy is a bit agressive given what it does...

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             0, concat_sub['is_poor_median']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_pushout_median.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + Mean Stacking
MinMax seems more gentle and it outperforms the previous one given its peformance score.

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_mean']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_mean.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + Median Stacking

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_median']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_median.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + BestBase Stacking

In [ ]:
# load the model with best base performance
sub_base = pd.read_csv(r"C:\Users\pvaish10\Desktop\csv_poverty\submission_X_G_tr2.csv")

In [ ]:
concat_sub['is_poor_base'] = sub_base['poor']
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_base']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')